## 卷积的API

In [1]:
import torch.nn as nn

nn.Conv2d()  
* 功能对多个二维的信号进行 二维卷积
* in_channels:输入通道数
* out_channels：输出通道数
* kernel_size:卷积核的尺寸
* stride:步长
* pandding:填充个数
* dilation: 空洞卷积大小
* groups：分组卷积设置
* bias:偏置

卷积后图的尺寸计算公式：  
简化版本：$$\cfrac{in_{size}+2*padding-kernel_{size}}{stride} + 1$$

nn.ConvTranspose2d()  
功能：转置卷积实现上采样
* in_channels:输入通道数
* out_channels:输出通道数
* kernel_size:卷积核尺寸
* stride：步长
* padding:填充个数
.......

尺寸计算公式：
$$out_{size} = (in_{size}-1)*stride+kernel_{size}-2*padding$$

池化层：最大池化，平均池化  
nn.Maxpool2d()  
* kernel_size:池化核的尺寸
* stride:步长
* padding:填充个数
* dilation:池化核间隔大小
* ceil_mode:尺寸向上取整
* return_indices：记录池化像素的索引

线性层：  
nn.Linear() 
* in_features:输入节点数
* out_features:输出节点数
* bias:是否需要偏置  
$$ y = xW^T +bias$$

### 创建模型，以Lenet为例子

In [3]:
class LeNet(nn.Module):
    def __init__(self, classes):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, classes)

    def forward(self, x):#拼接子模块
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

### 模型容器

In [6]:
class Lenet(nn.Module):
    def __init__(self,classes):
        super(Lenet,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc = nn.Sequential(
            nn.Linear(16*5*5,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,classes)
        )
    def forward(self,x):
        x=self.conv(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x

为顺序容器来命名：

In [21]:
from collections import OrderedDict

In [22]:
class Lenet_Dict(nn.Module):
    def __init__(self,classes):
        super(Lenet_Dict,self).__init__()
        self.conv = nn.Sequential(OrderedDict({
            'conv1':nn.Conv2d(3, 6, 5),
            'relu1':nn.ReLU(),
            'pool1':nn.MaxPool2d(2,2),
            'conv2':nn.Conv2d(6, 16, 5),
            'relu2':nn.ReLU(),
            'pool1':nn.MaxPool2d(2,2)
        }))
        self.fc = nn.Sequential(OrderedDict({
            'fc1':nn.Linear(16*5*5,120),
            'relu3':nn.ReLU(),
            'fc2':nn.Linear(120,84),
            'relu4':nn.ReLU(),
            'fc3':nn.Linear(84,classes)})
            
        )
    def forward(self,x):
        x=self.conv(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x

In [23]:
mode=Lenet_Dict(2)

In [24]:
mode

Lenet_Dict(
  (conv): Sequential(
    (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (relu2): ReLU()
  )
  (fc): Sequential(
    (fc1): Linear(in_features=400, out_features=120, bias=True)
    (relu3): ReLU()
    (fc2): Linear(in_features=120, out_features=84, bias=True)
    (relu4): ReLU()
    (fc3): Linear(in_features=84, out_features=2, bias=True)
  )
)

In [55]:
mode._modules.items()

odict_items([('conv', Sequential(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
)), ('fc', Sequential(
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (relu4): ReLU()
  (fc3): Linear(in_features=84, out_features=2, bias=True)
))])

In [69]:
mode._modules['conv']._modules['conv1']

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))

In [70]:
mode1 = LeNet(2)